użyj experiment.ipynb

## Import packages and modules

In [1]:
from models.tda_models import TDA_PI34_Model, TDA_PI42_Model, VECTOR_STITCHING_PI_Model_34, VECTOR_STITCHING_PI_Model_42
from models.raw_models import Raw_Model, Dummy_Model
import tensorflow as tf


## Transform raw data to persistance images and stitched RAW-PI images

In [2]:
from data_preprocessing import process_mnist_alike
(X, y), _ = tf.keras.datasets.mnist.load_data()

data = process_mnist_alike(data=(X, y), training_indices=(400,650), testing_indices=(100,200), dist_ratio=10)

c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:50: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_train_noisy = random_noise(X_train, mode="s&p",amount=0.05, seed=666)
c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:51: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_test_noisy = random_noise(X_test, mode="s&p",amount=0.05, seed=666)


1/8
2/8
3/8
4/8
5/8
6/8
7/8
8/8


# Initialize models

In [3]:
# models

raw_model = Raw_Model() # cnn models working on raw images
dummy_model = Dummy_Model() # fully dense model working on raw images
tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
tda_model_42 = TDA_PI42_Model() # cnn model working on persistance images
vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images
vector_stitching_model_42 = VECTOR_STITCHING_PI_Model_42() # cnn model working on stitched raw and PI images


# Fitting and validating models


In [7]:
tf.keras.backend.clear_session()

In [8]:
#TDA model
#early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

tda_model_34.model.fit(data["X_tr_tda_34"], data["y_tr_tda_34"], epochs=30, batch_size=1, validation_data=(data["X_test_noisy_tda_good_34"], data["y_test"]), callbacks=[early_stopping])

Epoch 1/30
250/250 [==============================] - 2s 7ms/step - loss: 0.0332 - accuracy: 0.9800 - val_loss: 3.0309 - val_accuracy: 0.4500
Epoch 2/30
250/250 [==============================] - 1s 6ms/step - loss: 0.0348 - accuracy: 0.9880 - val_loss: 5.7455 - val_accuracy: 0.4700
Epoch 3/30
250/250 [==============================] - 1s 6ms/step - loss: 0.0332 - accuracy: 0.9840 - val_loss: 5.0303 - val_accuracy: 0.4500
Epoch 4/30
250/250 [==============================] - 1s 6ms/step - loss: 0.0865 - accuracy: 0.9720 - val_loss: 2.7071 - val_accuracy: 0.4200
Epoch 5/30
250/250 [==============================] - 1s 6ms/step - loss: 0.5858 - accuracy: 0.9240 - val_loss: 2.4294 - val_accuracy: 0.4600
Epoch 6/30
250/250 [==============================] - 1s 6ms/step - loss: 0.4081 - accuracy: 0.9000 - val_loss: 9.8487 - val_accuracy: 0.2200
Epoch 7/30
250/250 [==============================] - 1s 6ms/step - loss: 0.1807 - accuracy: 0.9400 - val_loss: 4.3124 - val_accuracy: 0.4400


In [9]:
tda_model_34.model.evaluate(data["X_test_noisy_tda_good_34"], data["y_test"])

4/4 [==============================] - 0s 6ms/step - loss: 5.7455 - accuracy: 0.4700


[5.745476722717285, 0.4699999988079071]

In [10]:
tf.keras.backend.clear_session()

In [11]:
# RAW model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)


raw_model.model.fit(data["X_tr"], data["y_tr"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_random"], data["y_test"]), callbacks=[early_stopping])

Epoch 1/20
250/250 [==============================] - 3s 6ms/step - loss: 6.0086 - accuracy: 0.3240 - val_loss: 2.2967 - val_accuracy: 0.1900
Epoch 2/20
250/250 [==============================] - 1s 5ms/step - loss: 1.1956 - accuracy: 0.6600 - val_loss: 2.2945 - val_accuracy: 0.1400
Epoch 3/20
250/250 [==============================] - 1s 5ms/step - loss: 1.0139 - accuracy: 0.7400 - val_loss: 2.2873 - val_accuracy: 0.1400
Epoch 4/20
250/250 [==============================] - 1s 5ms/step - loss: 0.4619 - accuracy: 0.8480 - val_loss: 2.2699 - val_accuracy: 0.2100
Epoch 5/20
250/250 [==============================] - 1s 5ms/step - loss: 0.5497 - accuracy: 0.8400 - val_loss: 2.2770 - val_accuracy: 0.1600
Epoch 6/20
250/250 [==============================] - 1s 5ms/step - loss: 0.6001 - accuracy: 0.8080 - val_loss: 2.2542 - val_accuracy: 0.1800
Epoch 7/20
250/250 [==============================] - 1s 5ms/step - loss: 0.4425 - accuracy: 0.8800 - val_loss: 2.2485 - val_accuracy: 0.1500
Epoch 

In [12]:
raw_model.model.evaluate(data["X_test_noisy_random"], data["y_test"])

4/4 [==============================] - 0s 5ms/step - loss: 2.2699 - accuracy: 0.2100


[2.2698519229888916, 0.20999999344348907]

In [14]:
tf.keras.backend.clear_session()

In [15]:
#Vector stitching model

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

vector_stitching_model_34.model.fit(data["X_tr_vector_stitching_34"], data["y_tr_vector_stitching_34"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_vector_stitching_good_34"], data["y_test"]), callbacks=[early_stopping])

Epoch 1/20
250/250 [==============================] - 3s 8ms/step - loss: 2.2391 - accuracy: 0.2640 - val_loss: 2.2222 - val_accuracy: 0.3000
Epoch 2/20
250/250 [==============================] - 2s 7ms/step - loss: 1.4548 - accuracy: 0.5760 - val_loss: 2.1231 - val_accuracy: 0.3400
Epoch 3/20
250/250 [==============================] - 2s 7ms/step - loss: 1.0680 - accuracy: 0.7440 - val_loss: 1.5631 - val_accuracy: 0.5200
Epoch 4/20
250/250 [==============================] - 2s 7ms/step - loss: 0.7168 - accuracy: 0.7960 - val_loss: 1.6060 - val_accuracy: 0.5600
Epoch 5/20
250/250 [==============================] - 2s 7ms/step - loss: 0.4532 - accuracy: 0.8800 - val_loss: 1.3342 - val_accuracy: 0.5500
Epoch 6/20
250/250 [==============================] - 2s 7ms/step - loss: 0.7160 - accuracy: 0.8800 - val_loss: 2.3214 - val_accuracy: 0.2300
Epoch 7/20
250/250 [==============================] - 2s 7ms/step - loss: 0.5090 - accuracy: 0.8840 - val_loss: 1.1870 - val_accuracy: 0.5900
Epoch 

In [16]:
vector_stitching_model_34.model.evaluate(data["X_test_noisy_vector_stitching_good_34"], data["y_test"])

4/4 [==============================] - 0s 8ms/step - loss: 1.0871 - accuracy: 0.7600


[1.087052345275879, 0.7599999904632568]